In [2]:
import pandas as pd
import sqlite3
import mysql.connector
import subprocess

# Preparação

In [3]:
# Carregamento dos dados do excel
data = pd.read_excel('base-de-dados_mapa_23122022 (3).xlsx', sheet_name='Painel')

# Nomes das colunas são colocados em letras minúsculas para evitar ambiguidades com as tabelas do banco de dados
data = data.rename(columns=excel_dict)
data['id_ibge'] = data.index
data['id_rais'] = data.index
data['id_onu'] = data.index
data['id_cnes'] = data.index
data['id_siconfi'] = data.index
data['id_inepme'] = data.index
excel_lower = [c.lower() for c in data.columns]
excel_dict = dict(zip(data.columns, excel_lower))
data.sample(5)

,cod-mun,ano,pop,popurb,poprural,dens,peaurb,pearur,ifdm _a,ifdmedu,...,pop10a14anos,pop15a19anos,pop20a29anos,pop30a39anos,pop40a49anos,pop50a59anos,pop60a69anos,pop70a79anos,pop80+,itgm
1696,1303536,2011,27926.0,NaN,NaN,NaN,NaN,NaN,0.7057,0.6217,...,3459.0,3002.0,4922.0,4120.0,3027.0,2159.0,1091.0,392.0,114.0,NaN
11509,2207850,2010,3663.0,1793.0,1870.0,NaN,730.0,877.0,0.5333,0.4336,...,436.0,379.0,623.0,471.0,457.0,346.0,226.0,147.0,92.0,NaN
48560,3517703,2017,21081.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1669.0,1694.0,3296.0,3347.0,2712.0,2234.0,1571.0,788.0,359.0,NaN
44660,3300951,2009,8839.0,NaN,NaN,NaN,NaN,NaN,0.6825,0.7099,...,712.0,667.0,1338.0,1251.0,1195.0,924.0,612.0,320.0,143.0,NaN
34801,3121803,2020,7609.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,391.0,429.0,1069.0,1164.0,1102.0,1081.0,769.0,511.0,304.0,NaN


In [18]:
# Conecta ao banco de dados usando a biblioteca SQLite3
conn = sqlite3.connect('mapa.db')

# Cria um cursos para execução de querries
cursor = conn.cursor()

# Adiciona o esquema desejado no banco de dados
with open('tabelas_novas.sql', 'r') as sql_file:
    sql_script = sql_file.read()

cursor.executescript(sql_script)
conn.commit()

cursor.execute("""SELECT name FROM sqlite_master  
  WHERE type='table';"""
)

cursor.fetchall()

[('IBGE',), ('RAIS',), ('SICONFI',), ('ONU',), ('CNES',), ('INEPME',)]

# Tabela 1 - IBGE

In [6]:
# Seleciona tudo da tabela IBGE
cursor.execute("SELECT * FROM IBGE" )

# Coleta os nomes das colunas da tabela IBGE do banco de dados
columns_ibge = [c[0] for c in cursor.description]
columns_ibge_lower = [c[0].lower() for c in cursor.description]

In [26]:
# Criação de um dicionário de equivalência entre a tabela do banco de dados e o Excel
# Parte do processo é manual, pois o relacionamento das tabelas que não estão no excel precisa ser feito caso a caso
IBGE_IBGE = dict(zip(columns_ibge_lower, columns_ibge))
IBGE_excel = dict(zip(columns_ibge_lower, columns_ibge_lower))
IBGE_excel['id_ibge']='id_ibge'
IBGE_excel['municipio']='cod-mun'
IBGE_excel['codmunicipio']='cod-mun'
IBGE_excel['populacao']='pop'
excel_IBGE =  {v: k for k, v in IBGE_excel.items()}

In [27]:
# Seleciona somente as colunas do dataframe que equivalem às das tabelas do IBGE
data_IBGE = data[IBGE_excel.values()]
data_IBGE.columns = IBGE_excel.keys()
data_IBGE = data_IBGE.rename(columns=IBGE_IBGE)
data_IBGE.sample(5)

,ID_IBGE,Ano,Municipio,CodMunicipio,Populacao,PIBPerC,VABInd,VABS,VABAgr,VABAdm,...,EFE_PEC,PROD_AN,PROD_AQUI,VPAqui,VPPec,VACAS,TRAT_AR,TRAT_PES,TRAT_VB,ESTAB_CAPU
48633,48633,2020,3518206,3518206,33100.0,1.100342e+09,185810651.0,4.648696e+07,225961636.0,146358448.0,...,..,..,NaN,NaN,195768,1840,NaN,NaN,NaN,..
64221,64221,2012,4218707,4218707,98412.0,2.502550e+09,594341044.0,1.311169e+09,29435739.0,296380074.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
41083,41083,2016,3159308,3159308,5054.0,5.496335e+07,1999598.0,1.831982e+07,11449711.0,20913833.0,...,..,..,..,75,18019,7850,NaN,NaN,NaN,NaN
16614,16614,2019,2410256,2410256,6437.0,1.537613e+08,64630274.0,3.207211e+06,9158266.0,42914277.0,...,..,..,NaN,NaN,124,138,NaN,NaN,NaN,NaN
72638,72638,2015,5100508,5100508,10704.0,8.943040e+07,2872458.0,1.901238e+07,15599936.0,48444074.0,...,..,..,..,4321,2343,2180,NaN,NaN,NaN,NaN


In [28]:
data_IBGE = data_IBGE.replace('..',None)
data_IBGE = data_IBGE.replace('...',None)

In [32]:
data_IBGE.to_sql('IBGE',conn, if_exists='append', index=False)

77980

In [33]:
with open('ibge_dump.sql', 'w') as f:
    for line in conn.iterdump():
        f.write(f'{line}\n')

# Tabela 2 - RAIS

In [65]:
# Seleciona tudo da tabela RAIS
cursor.execute("SELECT * FROM RAIS" )

# Coleta os nomes das colunas da tabela IBGE do banco de dados
columns_rais = [c[0] for c in cursor.description]
columns_rais_lower = [c[0].lower() for c in cursor.description]
print(columns_rais)

['ID_RAIS', 'ID_IBGE', 'TxESF', 'TxESA', 'RMT', 'NE']


In [66]:
for c in columns_rais_lower:
    if c not in excel_lower:
        print(c)

In [67]:
# Criação de um dicionário de equivalência entre a tabela do banco de dados e o Excel
# Parte do processo é manual, pois o relacionamento das tabelas que não estão no excel precisa ser feito caso a caso
RAIS_RAIS = dict(zip(columns_rais_lower, columns_rais))
RAIS_excel = dict(zip(columns_rais_lower, columns_rais_lower))
excel_RAIS =  {v: k for k, v in RAIS_excel.items()}

In [68]:
# Seleciona somente as colunas do dataframe que equivalem às das tabelas do IBGE
data_RAIS = data[RAIS_excel.values()]
data_RAIS.columns = RAIS_excel.keys()
data_RAIS = data_RAIS.rename(columns=RAIS_RAIS)
data_RAIS.sample(5)

,ID_RAIS,ID_IBGE,TxESF,TxESA,RMT,NE
75089,75089,75089,NaN,NaN,NaN,NaN
32193,32193,32193,NaN,NaN,NaN,NaN
40055,40055,40055,NaN,NaN,NaN,NaN
9892,9892,9892,NaN,NaN,NaN,NaN
63948,63948,63948,NaN,NaN,NaN,NaN


In [69]:
data_RAIS = data_RAIS.replace('..',None)
data_RAIS = data_RAIS.replace('...',None)

In [71]:
data_RAIS.to_sql('RAIS',conn, if_exists='append', index=False)

77980

# Tabela 3 - SICONFI

In [72]:
# Seleciona tudo da tabela SICONFI
cursor.execute("SELECT * FROM SICONFI" )

# Coleta os nomes das colunas da tabela IBGE do banco de dados
columns_siconfi = [c[0] for c in cursor.description]
columns_siconfi_lower = [c[0].lower() for c in cursor.description]
print(columns_siconfi)

['ID_SICONFI', 'ID_IBGE', 'BO', 'GPRCL', 'TxIDT', 'GEdu', 'RCLPerC', 'GPerCEd', 'GPerCAS', 'GPerCS', 'GPerCT_A', 'DP', 'TRIB', 'PROP_TRAN']


In [73]:
for c in columns_siconfi_lower:
    if c not in excel_lower:
        print(c)

In [74]:
# Criação de um dicionário de equivalência entre a tabela do banco de dados e o Excel
# Parte do processo é manual, pois o relacionamento das tabelas que não estão no excel precisa ser feito caso a caso
SICONFI_SICONFI = dict(zip(columns_siconfi_lower, columns_siconfi))
SICONFI_excel = dict(zip(columns_siconfi_lower, columns_siconfi_lower))
excel_SICONFI =  {v: k for k, v in SICONFI_excel.items()}

In [76]:
# Seleciona somente as colunas do dataframe que equivalem às das tabelas do IBGE
data_SICONFI = data[SICONFI_excel.values()]
data_SICONFI.columns = SICONFI_excel.keys()
data_SICONFI = data_SICONFI.rename(columns=SICONFI_SICONFI)
data_SICONFI.sample(5)

,ID_SICONFI,ID_IBGE,BO,GPRCL,TxIDT,GEdu,RCLPerC,GPerCEd,GPerCAS,GPerCS,GPerCT_A,DP,TRIB,PROP_TRAN
55666,55666,55666,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
19104,19104,19104,0.938914,47.424495,4.395755,35.169494,3564.985546,1059.844081,81.107682,528.653753,3013.532333,1.690676e+06,222863884.51,2.394291e+08
52616,52616,52616,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7701,7701,7701,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8236,8236,8236,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [77]:
data_SICONFI = data_SICONFI.replace('..',None)
data_SICONFI = data_SICONFI.replace('...',None)

In [78]:
data_SICONFI.to_sql('SICONFI',conn, if_exists='append', index=False)

77980

# Tabela 4 - ONU

In [79]:
# Seleciona tudo da tabela ONU
cursor.execute("SELECT * FROM ONU" )

# Coleta os nomes das colunas da tabela IBGE do banco de dados
columns_onu = [c[0] for c in cursor.description]
columns_onu_lower = [c[0].lower() for c in cursor.description]
print(columns_onu)

['ID_ONU', 'ID_IBGE', 'IDHM']


In [80]:
for c in columns_onu_lower:
    if c not in excel_lower:
        print(c)

In [81]:
# Criação de um dicionário de equivalência entre a tabela do banco de dados e o Excel
# Parte do processo é manual, pois o relacionamento das tabelas que não estão no excel precisa ser feito caso a caso
ONU_ONU = dict(zip(columns_onu_lower, columns_onu))
ONU_excel = dict(zip(columns_onu_lower, columns_onu_lower))
excel_ONU =  {v: k for k, v in ONU_excel.items()}

In [82]:
# Seleciona somente as colunas do dataframe que equivalem às das tabelas do IBGE
data_ONU = data[ONU_excel.values()]
data_ONU.columns = ONU_excel.keys()
data_ONU = data_ONU.rename(columns=ONU_ONU)
data_ONU.sample(5)

,ID_ONU,ID_IBGE,IDHM
68494,68494,68494,NaN
53413,53413,53413,NaN
32562,32562,32562,NaN
57095,57095,57095,NaN
11055,11055,11055,NaN


In [85]:
data_ONU = data_ONU.replace('..',None)
data_ONU = data_ONU.replace('...',None)

In [ ]:
data_ONU.to_sql('ONU',conn, if_exists='append', index=False)

# Tabela 5 - CNES

In [87]:
# Seleciona tudo da tabela CNES
cursor.execute("SELECT * FROM CNES" )

# Coleta os nomes das colunas da tabela IBGE do banco de dados
columns_cnes = [c[0] for c in cursor.description]
columns_cnes_lower = [c[0].lower() for c in cursor.description]
print(columns_cnes)

['ID_CNES', 'ID_IBGE', 'TxEst', 'TxLint', 'TxMedSUS']


In [88]:
for c in columns_cnes_lower:
    if c not in excel_lower:
        print(c)

In [89]:
# Criação de um dicionário de equivalência entre a tabela do banco de dados e o Excel
# Parte do processo é manual, pois o relacionamento das tabelas que não estão no excel precisa ser feito caso a caso
CNES_CNES = dict(zip(columns_cnes_lower, columns_cnes))
CNES_excel = dict(zip(columns_cnes_lower, columns_cnes_lower))
excel_CNEs =  {v: k for k, v in CNES_excel.items()}

In [90]:
# Seleciona somente as colunas do dataframe que equivalem às das tabelas do IBGE
data_CNES = data[CNES_excel.values()]
data_CNES.columns = CNES_excel.keys()
data_CNES = data_CNES.rename(columns=CNES_CNES)
data_CNES.sample(5)

,ID_CNES,ID_IBGE,TxEst,TxLint,TxMedSUS
75281,75281,75281,NaN,NaN,NaN
8788,8788,8788,NaN,NaN,NaN
64163,64163,64163,NaN,NaN,NaN
75400,75400,75400,NaN,NaN,NaN
64700,64700,64700,NaN,NaN,NaN


In [91]:
data_CNES = data_CNES.replace('..',None)
data_CNES = data_CNES.replace('...',None)

In [92]:
data_CNES.to_sql('CNES',conn, if_exists='append', index=False)

77980

# Tabela 6 - INEPME

In [93]:
# Seleciona tudo da tabela INEPME
cursor.execute("SELECT * FROM INEPME" )

# Coleta os nomes das colunas da tabela IBGE do banco de dados
columns_inepme = [c[0] for c in cursor.description]
columns_inepme_lower = [c[0].lower() for c in cursor.description]
print(columns_inepme)

['ID_INEPME', 'ID_IBGE', 'IdbeAI', 'Dist_AI', 'Dist_AF', 'Dist_EM', 'MatTIAI', 'MatTIAF', 'MATTIEM']


In [94]:
for c in columns_inepme_lower:
    if c not in excel_lower:
        print(c)

In [95]:
# Criação de um dicionário de equivalência entre a tabela do banco de dados e o Excel
# Parte do processo é manual, pois o relacionamento das tabelas que não estão no excel precisa ser feito caso a caso
INEPME_INEPME = dict(zip(columns_inepme_lower, columns_inepme))
INEPME_excel = dict(zip(columns_inepme_lower, columns_inepme_lower))
excel_INEPME =  {v: k for k, v in INEPME_excel.items()}

In [96]:
# Seleciona somente as colunas do dataframe que equivalem às das tabelas do IBGE
data_INEPME = data[INEPME_excel.values()]
data_INEPME.columns = INEPME_excel.keys()
data_INEPME = data_INEPME.rename(columns=INEPME_INEPME)
data_INEPME.sample(5)

,ID_INEPME,ID_IBGE,IdbeAI,Dist_AI,Dist_AF,Dist_EM,MatTIAI,MatTIAF,MATTIEM
10114,10114,10114,4.1,34.8,55.6,69.3,NaN,NaN,NaN
4258,4258,4258,4.0,32.6,42.3,50.7,NaN,NaN,NaN
77644,77644,77644,4.0,22.6,32.1,51.7,NaN,NaN,NaN
8827,8827,8827,NaN,18.0,40.9,42.2,NaN,NaN,NaN
74997,74997,74997,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [97]:
data_INEPME = data_INEPME.replace('..',None)
data_INEPME = data_INEPME.replace('...',None)

In [98]:
data_INEPME.to_sql('INEPME',conn, if_exists='append', index=False)

77980

# Colunas sem tabelas no schema:

In [101]:
# Cria lista com todas as colunas de todas as tabelas e verifica faltantes
columns_schema = pd.unique([*columns_lower, *columns_rais_lower, *columns_inepme_lower, *columns_cnes_lower, *columns_onu_lower, *columns_siconfi_lower])
faltantes = ['id_ibge']
for c in excel_lower:
    if c not in columns_schema:
        faltantes.append(c)
        print(c, end='  ')

cod-mun  pop  dens  ifdm _a  ifdmedu   ifdms   ifdmer   idbeaf  idbeem  dist_isef  txps  txmedab  trans  ppopcadu  ppoppexpcadu  pfrarcadu  pfbf  p.s  gsau  gpercagr  cme  cmas  cmsa  fnde  vaagrfam  pnae  alt  bioma  relevo  solo  coleta_rs  agua  coleta_s  esgoto  esgoto_t  unnamed: 82  imp  exp  cambio  selic  ipca  cred_r  seg_r  proagro  arc  qntp  vp  rmp  sem_mud  vend_est  pop0a4anos  pop5a9anos  pop10a14anos  pop15a19anos  pop20a29anos  pop30a39anos  pop40a49anos  pop50a59anos  pop60a69anos  pop70a79anos  pop80+  itgm  

# Criação do dump da base de dados em formato sql

In [ ]:
with open('ibge_dump.sql', 'w') as f:
    for line in conn.iterdump():
        f.write(f'{line}\n')
        
cursor.close()
conn.close()